(sec:ml)=
# Machine learning

[Machine learning](https://en.wikipedia.org/wiki/Machine_learning) (ML) is a widely applied tool from computer science, ranging from mathematics over engineering to natural and even social sciences. 

Due to their simple nature and vast possibilities for their application, multiple ML frameworks exist that enable you to build and train a model with just a few lines of code. We will illustrate things with the classic example of identifying digits in hand-written images. The two ML frameworks we have adopted for this task are among the most used ones:

- [ML with PyTorch](sec:ml_pytorch)
- [ML with sklearn](sec:ml_scikit)